LOADING THE PACKAGES

In [21]:
#%pip install langchain
#%pip install faiss-cpu
#%pip install openai

In [22]:
#%pip install langchain unstructured
#%pip install tiktoken


DATASET PREPARATION

In [35]:
import os
os.environ["OPENAI_API_KEY"] = "#####"

In [4]:
urls = [
    'https://www.totalhealthandfitness.com/what-is-mindful-eating-your-complete-guide-to-enjoying-your-food-more/'
    'https://www.totalhealthandfitness.com/nutrition-and-chronic-disease/'
    'https://www.glofox.com/blog/fitness-blogs/'
]

In [5]:
from langchain.document_loaders import UnstructuredURLLoader
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()

In [6]:
data

[Document(page_content='403 Forbidden', metadata={'source': 'https://www.totalhealthandfitness.com/what-is-mindful-eating-your-complete-guide-to-enjoying-your-food-more/https://www.totalhealthandfitness.com/nutrition-and-chronic-disease/https://www.glofox.com/blog/fitness-blogs/'})]

In [7]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(separator='\n', 
                                      chunk_size=1000, 
                                      chunk_overlap=200)


docs = text_splitter.split_documents(data)

In [8]:
docs

[Document(page_content='403 Forbidden', metadata={'source': 'https://www.totalhealthandfitness.com/what-is-mindful-eating-your-complete-guide-to-enjoying-your-food-more/https://www.totalhealthandfitness.com/nutrition-and-chronic-disease/https://www.glofox.com/blog/fitness-blogs/'})]

In [9]:
len(docs)

1

In [10]:
import pickle
import faiss
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [34]:
vectorStore_openAI = FAISS.from_documents(docs, embeddings)

with open("faiss_store_openai.pkl", "wb") as f:
     pickle.dump(vectorStore_openAI, f)

with open("faiss_store_openai.pkl", "rb") as f:
    VectorStore = pickle.load(f)



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [14]:
VectorStore

NameError: name 'VectorStore' is not defined

MODEL FINE-TUNING
(GPT-3)

In [15]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

In [16]:
from langchain_community.chat_models import ChatOpenAI
llm=OpenAI(temperature=0, model_name='gpt-3.5-turbo-0125')

c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\llms\openai.py:249: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain_community\llms\openai.py:1070: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain_community.chat_models import ChatOpenAI`
  warnings.warn(


In [17]:
llm

OpenAIChat(client=APIRemovedInV1Proxy, model_name='gpt-3.5-turbo-0125', model_kwargs={'temperature': 0})

CHATBOT INTERFACE & TESTING

In [18]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=VectorStore.as_retriever())

NameError: name 'VectorStore' is not defined

In [19]:
chain({"question": "how to maintain a healthy diet?"}, return_only_outputs=True)

NameError: name 'chain' is not defined

API ENDPOINTS

In [23]:
#%pip install requests

In [24]:
import requests

In [27]:
def Questions(query):
    url = "http://localhost:11434/api/generate"
    params = {"query":query}
    response = requests.get(url, params=params)
    
    if response.status_code == 200:
        return response.json()
    
    else:
        print("Ollama API request failed.")
    

In [29]:
def Answer(question):
    answer = chain({"questions":question}, return_only_outputs=True)
    
    return answer

In [30]:
question = "how to maintain healthy diet?"
answer = Answer(question)
print(answer)

NameError: name 'chain' is not defined